In [ ]:
!pip install tmdbsimple

In [ ]:
import json
with open('/Users/purvikansara//.secret/tmdb-apikey.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']


In [ ]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)



In [ ]:
## movie objects have a .info dictionary 
info = movie.info()
info

In [ ]:
info['budget']


In [ ]:
info['revenue']


In [ ]:
info['imdb_id']



In [ ]:
movie = tmdb.Movies('tt1361336')
info = movie.info()
info['budget']


In [ ]:
# example from package README
response = movie.releases()
for c in movie.countries:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])


In [ ]:
# Get the movie object for the current id
movie = tmdb.Movies('tt1361336')
# save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()
# Loop through countries in releases
for c in releases['countries']:
    # if the country abbreviation==US
    if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']


In [ ]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies('tt1361336')
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
           movie_info['certification'] = c['certification']
    return movie_info

In [ ]:
test = get_movie_with_rating("tt0848228") #put your function name here
test

In [ ]:
import os, time,json
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)




In [ ]:
# what do we need to do actually
# 1. API call for years


In [ ]:
YEARS_TO_GET = [2000,2001]



In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):

In [ ]:
YEAR =2000

In [ ]:
#Defining the JSON file to store results for year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

In [ ]:
# Check if file exists
file_exists = os.path.isfile(JSON_FILE)

In [ ]:
# If it does not exist: create it
if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
with open(JSON_FILE,'w') as f:
json.dump([{'imdb_id':0}],f)

In [ ]:
# Load in the dataframe from project part 1 as basics:
import pandas as pd
basics = pd.read_csv('/Users/purvikansara/Documents/Purvi/Coding/TestingOnly/Week2/Project/Data/title_basics_cleaned.csv.gz')

basics.head()

In [ ]:
#Saving new year as the current df
df = basics.loc[ basics['startYear']==YEAR].copy()

In [ ]:
# saving movie ids to list
movie_ids = df['tconst'].copy()#.to_list()
movie_ids

In [ ]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)

previous_df

In [ ]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [ ]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [ ]:
#Get index and movie id from list
# INNER Loop

for movie_id in tqdm_notebook(movie_ids_to_get,
                              desc=f'Movies from {YEAR}',
                              position=1,
                              leave=True):
    # Attempt to retrieve then data for the movie id
    try:
        temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)

    # If it fails,  make a dict with just the id and None for certification.
    except Exception as e:
        continue

In [ ]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)